In [80]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from itertools import product
import os

from nlme import NonlinearMixedEffectsModel


ModuleNotFoundError: No module named 'nlme'

In [79]:

# Define function to convert parameters
def convert_params(a, b):
    A = -np.log(10 / a - 1)
    B = -np.log(500 / b - 1)
    return A, B

def convert_params_reverse(A, B):
    a = 10 / (1 + np.exp(-A))
    b = 500 / (1 + np.exp(-B))
    return a, b

# Define Weibull distribution function
def weibull_func(x, A, B):
    a, b = convert_params_reverse(A, B)
    return (a / b) * ((x / b) ** (a - 1)) * np.exp(- (x / b) ** a)

# Define function to fit Weibull model
def fit_weibull_model(xy_data_filt, start_params):
    def objective(params):
        A, B = params
        y_pred = weibull_func(xy_data_filt['WinMean'], A, B)
        return np.sum((y_pred - xy_data_filt['count_scaled']) ** 2)

    result = minimize(objective, start_params, method='Nelder-Mead')
    return result.x


In [56]:
xy_data_filt

,distance_window,Patient_ID,phenotype_combo,phenotype_from,phenotype_to,count,bin,WinMean,count_scaled,n_from,n_to
0,3.0,LUAD_D001,B cell_to_B cell,B cell,B cell,0.0,1.0,3.0,0.000000,32.0,32.0
1,3.0,LUAD_D002,B cell_to_B cell,B cell,B cell,0.0,1.0,3.0,0.000000,104.0,104.0
2,3.0,LUAD_D003,B cell_to_B cell,B cell,B cell,2.0,1.0,3.0,0.003883,126.0,126.0
3,3.0,LUAD_D004,B cell_to_B cell,B cell,B cell,0.0,1.0,3.0,0.000000,158.0,158.0
4,3.0,LUAD_D005,B cell_to_B cell,B cell,B cell,17.0,1.0,3.0,0.010291,403.0,403.0
...,...,...,...,...,...,...,...,...,...,...,...
109885,299.0,LUAD_D412,B cell_to_B cell,0,0,0.0,0.0,0.0,0.000000,0.0,0.0
109886,299.0,LUAD_D413,B cell_to_B cell,0,0,0.0,0.0,0.0,0.000000,0.0,0.0
109887,299.0,LUAD_D414,B cell_to_B cell,0,0,0.0,0.0,0.0,0.000000,0.0,0.0
109888,299.0,LUAD_D415,B cell_to_B cell,0,0,0.0,0.0,0.0,0.000000,0.0,0.0


In [7]:
path = os.getcwd()

# Load spatial data
# spatial_data_file = 'data/test_spatial_data_ovarian__processed__25subset.tsv'
firstNNhistogram_coordinates = f'{path}/results/step1_1nn_output.csv'
initial_params_file = f'{path}/results/step2_weibull_initial_params.csv'
output_file = f'{path}/results/step3_fittednlme_weibull_params.csv'
# output_path = f"{path}/Data/distances/"
input_path = f"{path}/Data/celldata/"
spatial_data_file = f"{input_path}celldata_20240506.csv"

dumdat = pd.read_csv(spatial_data_file, sep=',')

# Load initial Weibull parameters
initial_params = pd.read_csv(initial_params_file, sep=',')

# Load xy data
xy_data_exc_oct = pd.read_csv(firstNNhistogram_coordinates, sep=',')
xy_data_exc_oct['distance_window'] = xy_data_exc_oct['WinMean']


In [10]:

# Define parameters
B_max = 500
A_max = 10
threshold_pixels = 300
threshold_microns = threshold_pixels

# Define phenotype combinations to exclude
# phenotype_combinations_to_exclude = ['Bcell_to_negative', 'Macrophage_to_T-cell', 'Bcell_to_Cancer']


In [50]:
xy_data_filt

,Patient_ID,phenotype_from,phenotype_to,count,bin,WinMean,count_scaled,phenotype_combo,distance_window,n_from,n_to
0,LUAD_D001,B cell,B cell,0,0,2.0,0.000000,B cell_to_B cell,2.0,32,32
1,LUAD_D001,B cell,B cell,0,1,3.0,0.000000,B cell_to_B cell,3.0,32,32
2,LUAD_D001,B cell,B cell,0,2,4.0,0.000000,B cell_to_B cell,4.0,32,32
3,LUAD_D001,B cell,B cell,2,3,5.0,0.015385,B cell_to_B cell,5.0,32,32
4,LUAD_D001,B cell,B cell,8,4,6.0,0.061538,B cell_to_B cell,6.0,32,32
...,...,...,...,...,...,...,...,...,...,...,...
109885,LUAD_D416,B cell,B cell,0,292,294.0,0.000000,B cell_to_B cell,294.0,152,152
109886,LUAD_D416,B cell,B cell,0,293,295.0,0.000000,B cell_to_B cell,295.0,152,152
109887,LUAD_D416,B cell,B cell,0,294,296.0,0.000000,B cell_to_B cell,296.0,152,152
109888,LUAD_D416,B cell,B cell,0,295,297.0,0.000000,B cell_to_B cell,297.0,152,152


In [66]:
xy_data_filt['Patient_ID'].unique()

array(['LUAD_D001', 'LUAD_D002', 'LUAD_D003', 'LUAD_D004', 'LUAD_D005',
       'LUAD_D006', 'LUAD_D007', 'LUAD_D008', 'LUAD_D009', 'LUAD_D010',
       'LUAD_D011', 'LUAD_D012', 'LUAD_D013', 'LUAD_D015', 'LUAD_D016',
       'LUAD_D019', 'LUAD_D020', 'LUAD_D021', 'LUAD_D022', 'LUAD_D023',
       'LUAD_D024', 'LUAD_D025', 'LUAD_D026', 'LUAD_D027', 'LUAD_D028',
       'LUAD_D029', 'LUAD_D030', 'LUAD_D031', 'LUAD_D032', 'LUAD_D033',
       'LUAD_D034', 'LUAD_D035', 'LUAD_D036', 'LUAD_D037', 'LUAD_D038',
       'LUAD_D039', 'LUAD_D040', 'LUAD_D041', 'LUAD_D042', 'LUAD_D043',
       'LUAD_D044', 'LUAD_D045', 'LUAD_D046', 'LUAD_D047', 'LUAD_D048',
       'LUAD_D049', 'LUAD_D050', 'LUAD_D051', 'LUAD_D052', 'LUAD_D053',
       'LUAD_D054', 'LUAD_D055', 'LUAD_D056', 'LUAD_D057', 'LUAD_D058',
       'LUAD_D059', 'LUAD_D060', 'LUAD_D061', 'LUAD_D062', 'LUAD_D063',
       'LUAD_D064', 'LUAD_D065', 'LUAD_D066', 'LUAD_D067', 'LUAD_D071',
       'LUAD_D072', 'LUAD_D073', 'LUAD_D074', 'LUAD_D075', 'LUAD

In [78]:
A

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 23.285459954289315
             x: [-2.197e+00  4.124e+01]
           nit: 90
          nfev: 179
 final_simplex: (array([[-2.197e+00,  4.124e+01],
                       [-2.197e+00,  4.124e+01],
                       [-2.197e+00,  4.124e+01]]), array([ 2.329e+01,  2.329e+01,  2.329e+01]))

In [76]:
n_cells = dumdat.groupby(['Patient_ID', 'celltype']).size().reset_index(name='n')
success_models = []
weib_coefs = pd.DataFrame(columns=['A','B','a', 'b', 'Patient_ID', 'phenotype_combo'])

# Iterate through different threshold minimum number of cells sample
for threshold_cells in [0, 20, 50, 70, 100]:
    model_objects = {}
    subset_combinations = xy_data_exc_oct['phenotype_combo'].unique()

    for combi in subset_combinations:
        print(threshold_cells)  
        xy_data_filt = xy_data_exc_oct[xy_data_exc_oct['phenotype_combo'] == combi].copy()

        xy_data_filt = xy_data_filt.merge(n_cells[['Patient_ID', 'celltype', 'n']],
                                          left_on=['Patient_ID', 'phenotype_from'], right_on=['Patient_ID', 'celltype'])
        xy_data_filt = xy_data_filt.merge(n_cells[['Patient_ID', 'celltype', 'n']],
                                          left_on=['Patient_ID', 'phenotype_to'], right_on=['Patient_ID', 'celltype'], suffixes=['_from', '_to']).drop(columns=['celltype_from', 'celltype_to'])

        xy_data_filt = xy_data_filt[(xy_data_filt['n_to'] > threshold_cells) & (xy_data_filt['n_from'] > threshold_cells)]

        # Add missing observations in data and set to 0
        xy_data_filt = pd.DataFrame(product([1] + xy_data_filt['distance_window'].unique(),
                                            xy_data_filt['Patient_ID'].unique(),
                                            [combi]), columns=['distance_window', 'Patient_ID', 'phenotype_combo']).merge(
            xy_data_filt, on=['distance_window', 'Patient_ID', 'phenotype_combo'], how='left').fillna(0)

        mean_params = initial_params[(initial_params['combo'] == combi) &
                                     (initial_params['Patient_ID'].isin(xy_data_filt['Patient_ID'].unique()))].groupby(
            'term')['estimate'].mean().reset_index()

        start_params = mean_params.set_index('term').loc[['shape', 'scale'], 'estimate'].tolist()

        fit_result = fit_weibull_model(xy_data_filt, start_params)

        # Store Weibull parameters
        A, B = fit_result
        a, b = convert_params_reverse(A, B)
        new_data_weib_param = pd.DataFrame({'A': [A], 'B': [B], 'sample': xy_data_filt['Patient_ID'].unique(),
                                            'phenotype_combo': [combi], 'a': [a], 'b': [b]})
        weib_coefs = pd.concat([weib_coefs, new_data_weib_param], ignore_index=True)

        # Store x/y coordinates of input/output model
        xy_data_weib_imcision = pd.concat([xy_data_weib_imcision, xy_data_filt.assign(yhat=weibull_func(xy_data_filt['x'], A, B))])

        model_objects[combi] = fit_result

    success_models.update({key: value for key, value in model_objects.items() if isinstance(value, tuple)})

# Process parameters
final_weibull_parameters = pd.DataFrame(columns=['a', 'b', 'sample', 'phenotype_combo'])
for combi, model in success_models.items():
    A, B = model
    fixefect = fixef(success_models[combi])
    ranef = ranef(success_models[combi])
    final_weibull_parameters = pd.concat([final_weibull_parameters,
                                           ranef.assign(Patient_ID=ranef.index, phenotype_combo=combi).
                                           assign(A=lambda x: x['A'] + fixefect['A']).
                                           assign(B=lambda x: x['B'] + fixefect['B']).
                                           assign(a=lambda x: 10 / (1 + np.exp(-x['A']))).
                                           assign(b=lambda x: 500 / (1 + np.exp(-x['B'])))], ignore_index=True)

# Save final Weibull parameters
# final_weibull_parameters.to_csv(output_file, sep='\t', index=False)


0


TypeError: bad operand type for unary -: 'OptimizeResult'